In [1]:
# 화면에서 검색어를 입력받아
# selenium으로 키워드 검색
# 검색결과를 수집( 단일 영상 타입)
# - 채널명, 영상제목, 영상url, 조회수, 영상등록날짜
# - 'h3',{'class':'title-and-badge style-scope ytd-video-renderer'}
# 검색결과를 수집( 재생목록 타입)
# - 'div',{'class':'style-scope ytd-playlist-renderer'}
# - 채널명, 재생목록제목, 재생목록url
# 데이터 프레임 만들기
# - 단일영상타입 데이터 프레임
# - 재생목록타입 데이터 프레임

In [100]:
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import pandas as pd

In [5]:
driver = wb.Chrome('C:/Users/학생/0.pjha/chromedriver')

In [6]:
driver.get('http://youtube.com')

In [7]:
driver.find_element_by_id('search').send_keys('selenium')

In [8]:
driver.find_element_by_id('search-icon-legacy').click()

In [9]:
source = driver.page_source

In [142]:
soup = bs(source, 'html.parser')

In [160]:
b = soup.find_all('ytd-item-section-renderer',{'class':'style-scope ytd-section-list-renderer'})

In [168]:
single_type = soup.find_all('h3',{'class':'title-and-badge style-scope ytd-video-renderer'}) 

In [161]:
single_type[0].find('a')['aria-label'].split(' 게')[0]

['파이썬 자동화 selenium 사용법 브라우저 매크로 프로그램 만들기',
 '시자: 프로그래머 김플 스튜디오 1년 전 12분 9초 조회수 35,596회']

In [164]:
single_type[0].find('a')['href']

'/watch?v=ZFmTwbRQ0uc'

In [115]:
len(single_type)

19

In [94]:
list_type = soup.find_all('div',{'class':'style-scope ytd-playlist-renderer'})
len(list_type)

5

In [99]:
searchlist_list_type_title = []
searchlist_list_type_channel = []
searchlist_list_type_url = []
for i in range(len(list_type)):
    searchlist_list_type_title.append(list_type[i].find('span')['title'])
    searchlist_list_type_channel.append(list_type[i].find('div',{'id':'tooltip'}).text.replace('\n','').replace(' ',''))
    list_type_url = 'https://www.youtube.com' + list_type[i].find('a')['href']
    searchlist_list_type_url.append(list_type_url)

In [101]:
list_df = pd.DataFrame(list(zip(searchlist_list_type_title,searchlist_list_type_channel,
                               searchlist_list_type_url)),columns=['리스트제목',
                                                                  '채널명','url'])
list_df

,리스트제목,채널명,url
0,Python Selenium Tutorials,TechWithTim,https://www.youtube.com/watch?v=Xjv1sY630Uc&li...
1,파이썬 selenium(셀레니움) 강의,프로그래머김플스튜디오,https://www.youtube.com/watch?v=j_BW5vNrcxA&li...
2,Python+Selenium入门教学,Inty,https://www.youtube.com/watch?v=o6yzNaRAzW8&li...
3,Selenium with Python,SDET-QAAutomationTechie,https://www.youtube.com/watch?v=IYILCEV5j6s&li...
4,SELENIUM WEBDRIVER TUTORIAL - Learn Selenium A...,SoftwareTestingMentor,https://www.youtube.com/watch?v=25U-KUpWd1c&li...


In [167]:
searchlist_single_type_channel = []
searchlist_single_type_title = []
searchlist_single_type_date = []
searchlist_single_type_clicks = []
searchlist_single_type_url = []
for i in range(len(single_type)):
    searchlist_single_type_channel.append(soup.find_all('div',{'id':'channel-info'})[i].find('a',{'class':'yt-simple-endpoint style-scope yt-formatted-string'}).text)
    searchlist_single_type_title.append(single_type[i].find('a')['aria-label'].split(' 게')[0])
    url = 'https://www.youtube.com'+single_type[i].find('a')['href']
    searchlist_single_type_url.append(url)
for i in range(0,38):
    if i%2 == 1:
        searchlist_single_type_date.append(b[1].find_all('span',{'class':'style-scope ytd-video-meta-block'})[i].text)
    else:
        searchlist_single_type_clicks.append(b[1].find_all('span',{'class':'style-scope ytd-video-meta-block'})[i].text)

single_df = pd.DataFrame(list(zip(searchlist_single_type_title,searchlist_single_type_channel,
                                searchlist_single_type_url,searchlist_single_type_date,searchlist_single_type_clicks)),
                             columns=['제목','채널','url','날짜','조회수'])
single_df

,제목,채널,url,날짜,조회수
0,파이썬 자동화 selenium 사용법 브라우저 매크로 프로그램 만들기,프로그래머 김플 스튜디오,https://www.youtube.com/watch?v=ZFmTwbRQ0uc,1년 전,조회수 3.5만회
1,Selenium 기초 끝내기 - 셀레니움과 웹 드라이버 설치부터 웹사이트 스크래핑 ...,이수안컴퓨터연구소,https://www.youtube.com/watch?v=dDEESB4Iw8g,10개월 전,조회수 2.7천회
2,떠오르는 영양제 셀레늄! 과연 내 몸에 득일까 독일까?,내과전문의 닥터케이 Dr.K,https://www.youtube.com/watch?v=1i96FPkqREk,4개월 전,조회수 4.4천회
3,[파이썬 매크로 만들기] 6. Selenium으로 홈페이지 맘대로 주무르기,초보코딩,https://www.youtube.com/watch?v=_gkhgS33QIc,1년 전,조회수 6.4천회
4,Selenium - 동적페이지 크롤링 (동적vs정적 페이지),생각하는코딩,https://www.youtube.com/watch?v=9H9aTzj00lg,1년 전,조회수 1.4천회
5,웹크롤링의 여정 - 5 Selenium,박성준,https://www.youtube.com/watch?v=i3zXATaxBHc,1년 전,조회수 4천회
6,"Python Selenium 튜토리얼 # 1-웹 스크랩 핑, 봇 및 테스트",Tech With Tim,https://www.youtube.com/watch?v=Xjv1sY630Uc&pp...,1년 전,조회수 53만회
7,Selenium Full Course - Learn Selenium in 12 Ho...,edureka!,https://www.youtube.com/watch?v=FRn5J31eAMw,1년 전,조회수 49만회
8,고급 Python 프로그래밍 : Selenium을 사용한 브라우저 자동화,TheCodex,https://www.youtube.com/watch?v=GJjMjB3rkJM&pp...,3년 전,조회수 31만회
9,파이썬 코딩 무료 강의 (기본편) - 6시간 뒤면 여러분도 개발자가 될 수 있어요 ...,나도코딩,https://www.youtube.com/watch?v=kWiCuklohdY,1년 전,조회수 189만회
